In [1]:
from select_stock_pool import *

In [2]:
date = 20191101
conn = pymysql.connect(**DB_INFO, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

In [3]:
import importlib
mod = importlib.import_module(f'select_stock_pool')
func_names = ['get_accural_info', 'get_construct_info', 'get_rd_account_info', 'get_capital_risk_info', 'get_manager_leave_info', 
              'get_inter_control_info', 'get_over_invest_info', 'get_related_trade_info', 'get_analyst_expect_info', 'get_holder_trade_info',
              'get_risk_event_info', 'get_finance_report_info']

res = []

for i, func_name in enumerate(func_names):
    func = getattr(mod, func_name)
    print('\n', i, func_name, pd.datetime.now())
    res.append(func(date, conn))
print('\n', pd.datetime.now())


 0 get_accural_info 2019-12-13 15:10:54.918763
Reading sql AShareIndustriesClassCITICS
Reading sql AShareBalanceSheet
Reading sql AShareIncome
Reading sql AShareCashFlow

 1 get_construct_info 2019-12-13 15:11:10.141776
Reading sql AShareIndustriesClassCITICS
Reading sql AShareBalanceSheet
Reading sql AShareCashFlow

 2 get_rd_account_info 2019-12-13 15:11:29.585777
Reading sql WindCustomCode
Reading sql AShareRDexpenditure
Reading sql AShareIncome
Reading sql AShareAuditOpinion

 3 get_capital_risk_info 2019-12-13 15:12:13.929471
Reading sql ASharePledgeproportion
Reading sql AShareCapitalization
Reading sql AEquFroPleInfoRepperend
Reading sql AShareFloatHolder

 4 get_manager_leave_info 2019-12-13 15:13:08.377816
Reading sql AShareManagement


/home/ywang/proj_negative_alpha/select_stock_pool.py:385: RuntimeWarning: invalid value encountered in long_scalars
  manager_leave_ratio = np.sum(sr_manager_leave) / np.sum(df.s_info_manager_type == 1)
/home/ywang/proj_negative_alpha/select_stock_pool.py:385: RuntimeWarning: divide by zero encountered in long_scalars
  manager_leave_ratio = np.sum(sr_manager_leave) / np.sum(df.s_info_manager_type == 1)



 5 get_inter_control_info 2019-12-13 15:13:19.964734
Reading sql AShareDescription
Reading sql AShareIssuingDatePredict
Reading sql AShareIllegality
Reading sql AShareRegInv
Reading sql AShareProsecution
Reading sql AShareProfitNotice


/home/ywang/proj_negative_alpha/select_stock_pool.py:490: RuntimeWarning: Mean of empty slice
  sr_unaccuracy_notice = 1.0 * ((sr_notice_lowerbound - sr_profit)/sr_notice_lowerbound).groupby('sid').apply(lambda x: np.nanmean(x))#连续型变量
/home/ywang/env/dev/lib/python3.6/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)



 6 get_over_invest_info 2019-12-13 15:13:41.131076
Reading sql AShareBalanceSheet
Reading sql MergerParticipant
Reading sql MergerEvent

 7 get_related_trade_info 2019-12-13 15:14:06.454698
Reading sql WindCustomCode
Reading sql AShareBalanceSheet
Reading sql AShareRalatedTrade
Reading sql AShareRelatedclaimsdebts
Reading sql MergerParticipant
Reading sql MergerEvent

 8 get_analyst_expect_info 2019-12-13 15:14:22.973300
Reading sql AShareStockRatingConsus
Reading sql AShareConsensusRollingData

 9 get_holder_trade_info 2019-12-13 15:14:50.047359
Reading sql AShareCapitalization
Reading sql AShareMjrHolderTrade

 10 get_risk_event_info 2019-12-13 15:14:53.914138
Reading sql MergerParticipant
Reading sql MergerEvent
Reading sql AShareCompRestricted
Reading sql AShareMajorEvent

 11 get_finance_report_info 2019-12-13 15:16:00.818191
Reading sql WindCustomCode
Reading sql AShareIncome
Reading sql AShareProfitNotice
Reading sql AShareAuditOpinion
Reading sql AShareGovernmentgrants

 2019-

In [ ]:
# for i, ele in enumerate(res2):
#     assert np.all(ele.index[:-1] < ele.index[1:])
#     # print(i+1, ele.name, ele.index[:5])

In [4]:
from data_tools.api import query_table, trade_days, is_trade_date, get_trade_dates
def get_stk_descr(conn):
    table_name = 'wind.ASHAREDESCRIPTION'
    cols_map = {'S_INFO_WINDCODE': 'sid', 'S_INFO_NAME': 'name',
                'S_INFO_LISTDATE': 'list_date', 'S_INFO_DELISTDATE': 'delist_date'}
    sql = f"SELECT %s FROM %s" % (', '.join(list(cols_map.keys())), table_name)
    df = pd.read_sql_query(sql, conn)
    df.rename(cols_map, axis=1, inplace=True)
    df = df.sort_values(by=['sid'], ascending=True)
    df = df[(df.sid.apply(lambda x: x is not None)) 
            & (df.sid.apply(lambda x: x[::8] in ['0Z', '3Z', '6H']))]
    df = df.replace([None], '99999999')
    df['list_date'] = df['list_date'].values.astype('i4')
    df['delist_date'] = df['delist_date'].values.astype('i4')
    return df.reset_index(drop=True)


def get_db_info(date, conn):
    start_date = get_prev_n_trade_date(date, 59)
    df_data = query_table('DailyBar', start_date, date, ['close', 'amount', 'IPODays', 'ST'])
    df_data.sort_values(['DataDate', 'sid'], inplace=True)
    
    unique_dates = np.sort(np.unique(df_data.DataDate.values))
    unique_sids = np.sort(np.unique(df_data.sid.values))
    dates_pos = np.searchsorted(unique_dates, df_data.DataDate.values)
    sids_pos = np.searchsorted(unique_sids, df_data.sid.values)
    m, n = len(unique_dates), len(unique_sids)
    assert m == 60
    
    d = dict()
    for fl in ['close', 'amount', 'IPODays', 'ST']:
        d[fl] = np.full([m, n], np.nan)
        d[fl][dates_pos, sids_pos] = df_data[fl].values

    cond_list = 1.0 * (d['IPODays'][-1] < 120) # 剔除的为1
    cond_st = 1.0 * (d['ST'][-1] == 1.0)
    cond_amount = np.full([n,], 0.0)
    for j in range(n):
        arr = d['amount'][:,j]
        arr2 = arr[~(np.isnan(arr)) & (arr>0)]
        if len(arr2) < 20:
            continue
        arr2 = arr2[-20:]
        pos = np.argmax(arr2)
        mean_amount = (np.sum(arr2) - arr2[pos]) / 19
        cond_amount[j] = 1.0 * (mean_amount < 2.e7)
    
    pos = np.searchsorted(dates_pos, dates_pos[-1])
    df_res = pd.DataFrame([], index=unique_sids[sids_pos[pos:]])
    df_res['list_days'] = cond_list[sids_pos[pos:]]
    df_res['st'] = cond_st[sids_pos[pos:]]
    df_res['amount'] = cond_amount[sids_pos[pos:]]
    df_res.index.name = 'sid'
    return df_res

In [5]:
from itertools import chain
res2 = list(chain(*res))

df_db = get_db_info(date, conn)
for ele in res2:
    ele2 = ele.reindex(df_db.index)
    df_db[ele2.name] = ele2.values

/home/ywang/env/dev/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in less


In [11]:
stock_pool=set()
for item in df_db.columns[3:]:
    if item=='issuing_delay':
        continue
    risk_item = df_db[item].dropna()
    if len(risk_item)==0:
        continue
    bound_value = np.percentile(risk_item,98)
    risk_stock=risk_item[risk_item>=bound_value].index
    print(item,len(risk_stock))
    stock_pool=stock_pool|set(risk_stock)

accural_profit 75
accural_abnormal 74
manipulate 71
construct_growth 65
rd_ratio 63
change_audit 543
pledge_ratio 72
fro_ple_ratio 54
top10_trust_ratio 74
top10_trust_num 76
cfo_leave 150
manager_leave 169
illegality 29
reginv 67
prosecution 28
unaccuracy_notice 55
goodwill_ratio 44
merger_times 57
related_sellbuy_ratio 31
related_debts_ratio 14
related_merger_ratio 6
related_trade_num 47
rating_delta 24
est_price_delta 21
eps_delta 38
holder_sell 11
manager_sell 8
merger_failure 27
restricted_ratio 2
bond_default 27
enquiry_letter 32
actual_deficit 450
notice_deficit 132
notice_collapse 101
audit_opinion 210
grants_ratio 53
